# Topic Modelling using NLP - Kaggle Job Descriptions Data

## Part 2 - EDA, Feature extraction and Topic Modelling using NLP 


In [1]:
import pandas as pd
import support_functions as sf

### Load Kaggle dataset

In [2]:
i_jobs = pd.read_csv('../offline-datasets/Indeed-jobs/IndeedJobs.csv', na_filter=False)

In [3]:
i_jobs.head()

,Unnamed: 0,Location,Company,Job Title,Job Description,Salary
0,1,London,Mango Business Solutions,Data Consultant,A data consultant experienced in working in a ...,No salary information
1,2,London,Cambridge Healthcare Research,Associate Consultant,Where appropriate manages external contractors...,No salary information
2,3,London,Celonis,Intern/Working Student Data Science / Business...,... have already gained substantial know-how i...,No salary information
3,4,London,Celonis SE,Intern/Working Student Data Science / Business...,... have already gained substantial know-how i...,No salary information
4,5,London,managementsolutions,DATA SCIENCE CONSULTANT LONDON,You will be working in key projects for leadin...,No salary information


In [4]:
# Drop 1st column
i_jobs.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
i_jobs.head()

,Location,Company,Job Title,Job Description,Salary
0,London,Mango Business Solutions,Data Consultant,A data consultant experienced in working in a ...,No salary information
1,London,Cambridge Healthcare Research,Associate Consultant,Where appropriate manages external contractors...,No salary information
2,London,Celonis,Intern/Working Student Data Science / Business...,... have already gained substantial know-how i...,No salary information
3,London,Celonis SE,Intern/Working Student Data Science / Business...,... have already gained substantial know-how i...,No salary information
4,London,managementsolutions,DATA SCIENCE CONSULTANT LONDON,You will be working in key projects for leadin...,No salary information


### Import Spacy Library

In [6]:
# Perform standard imports
import spacy
nlp = spacy.load('en_core_web_sm')

In [7]:
# Import the displaCy library
from spacy import displacy

### Transforming Job Descriptions to NLP docs using Spacy

#### Kaggle Job Descriptions

In [8]:
i_jobs.isnull().sum()

Location           0
Company            0
Job Title          0
Job Description    0
Salary             0
dtype: int64

In [9]:
len(i_jobs['Job Description'])

75

In [10]:
len(i_jobs['Job Description'][:1000])

75

In [11]:
nlp_jd_i = []
for jd in i_jobs['Job Description'][:50]:
    nlp_jd_i.append(nlp(jd))

In [12]:
for i in nlp_jd_i:
    sf.show_entities(i)
    print('\n\n')


 - GPE - Countries, cities, states




 - GPE - Countries, cities, states



Business Intelligence - ORG - Companies, agencies, institutions, etc.

 - GPE - Countries, cities, states



Business Intelligence - ORG - Companies, agencies, institutions, etc.

 - GPE - Countries, cities, states



Discovery - PRODUCT - Objects, vehicles, foods, etc. (not services)
Simulation - WORK_OF_ART - Titles of books, songs, etc.



No named entities found.




 - GPE - Countries, cities, states
GreySpark - GPE - Countries, cities, states
Data Practice - ORG - Companies, agencies, institutions, etc.



100% - PERCENT - Percentage, including "%"
HMRC - ORG - Companies, agencies, institutions, etc.



Capgemini Invent - ORG - Companies, agencies, institutions, etc.



No named entities found.



No named entities found.



OC&C - ORG - Companies, agencies, institutions, etc.



2:1 - CARDINAL - Numerals that do not fall under another type
STEM - ORG - Companies, agencies, institutions, etc.

 - GPE - 

In [13]:
for i in nlp_jd_i:
    for sent in i.sents:
        docx = nlp(sent.text)
        if docx.ents:
            displacy.render(docx, style='ent', jupyter=True)
#            doc, style='ent', options=options
        else:
            print(docx.text)
        print('\n')
    print('\n\n')



Good knowledge of R and/or Python and the surrounding data environments.









You extract and transform customers' data and design process- and…









You extract and transform customers’ data and design process- and…












Experience of the banking environment, products and front and back office operations/ functions;Experience of data analytics and visualisation products such as…












You will be working in key projects for leading organizations in the strategy, commercial effectiveness, transformation: organization and processes, risk…





Strong analytical capabilities, familiarity and comfort with business, scientific and clinical data (essential).







...






A strong academic background in social sciences (e.g. economics, political science, public administration, economic geography).









The candidate is expected to understand key data sources, perform and synthesize complex…









Deep technical expertise in data science, statistical modelling and/or machine learning.









Using research from competitive pipelines, company websites, prescribing information,…









You will be working in key projects for leading organizations in IT…







In [14]:
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

## Pre-processing using skicit-learn and Visualisation Libraries

In [15]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig

#### Configuration 

In [16]:
# Number of points to use from Dataset
n_samples = 75

# Number of features from vectorised featureset
n_features = 1000

# Number of Topics to extract
n_components = 10

# Number of Words per topic
n_top_words = 15

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Use Count Vectorizer (Bag of Words) for extracting features

In [17]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
tf = tf_vectorizer.fit_transform(i_jobs['Job Description'][:n_samples])

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
tf

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<75x191 sparse matrix of type '<class 'numpy.int64'>'
	with 713 stored elements in Compressed Sparse Row format>

#### Use Tf-IDF Vectorizer for extracting features

In [21]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
tfidf = tfidf_vectorizer.fit_transform(i_jobs['Job Description'][:n_samples])

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
tfidf

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<75x191 sparse matrix of type '<class 'numpy.float64'>'
	with 713 stored elements in Compressed Sparse Row format>

## LDA modelling

#### Kaggle Job Descriptions

In [24]:
LDA = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# Fit document using Bag of Words (Term Frequency)
LDA1 = LDA.fit(tf)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [129]:
# Fit again using Inverse Document Frequency (TFIDF)
#LDA2 = LDA.fit(tfidf)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# Check features length
len(tf_vectorizer.get_feature_names())

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


191

In [27]:
# Check vectors length
#len(tfidf_vectorizer.get_feature_names())

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
import random

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
for i in range(10):
    random_word_id = random.randint(0,n_features)
    print(tf_vectorizer.get_feature_names()[random_word_id])

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndexError: list index out of range

#### Check number of components

In [30]:
len(LDA1.components_)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


10

In [31]:
#len(LDA2.components_)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Top 10 words per topic Using Bag of Words (TF - Document Term Frequency)

In [32]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP {n_top_words} WORDS FOR TOPIC #{index}')
    print([tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-n_top_words:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['know', 'analysis', 'statistical', 'non', 'principles', 'assessing', 'submissions', 'career', 'customer', 'solutions', 'providing', 'graduate', 'experience', 'equip', 'skills']


THE TOP 15 WORDS FOR TOPIC #1
['scientific', 'management', 'experience', 'preferred', 'science', 'strategy', 'life', 'sciences', 'engineering', 'delivering', 'customer', 'degree', 'marketing', 'business', 'data']


THE TOP 15 WORDS FOR TOPIC #2
['non', 'engineering', 'understanding', 'strengths', 'skills', 'range', 'encourage', 'cultivate', 'wide', 'areas', 'software', 'consultants', 'technical', 'science', 'data']


THE TOP 15 WORDS FOR TOPIC #3
['gained', 'customers', 'intelligence', 'creatively', 'turn', 'insights', 'apply', 'develop', 'analytic', 'knowledge', 'approaches', 'science', 'results', 'information', 'data']


THE TOP 15 WORDS FOR TOPIC #4
['minimum', 'structured', 'techniques', 'optionally', 'modeling', 'sources', 'processing', 'driven', 'work', 'consultant', 'ensur

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(LDA, tf, tf_vectorizer, mds='tsne')
dash

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/pandas/core/common.py:471: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not issubclass(into, collections.Mapping):


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5      32.420952   8.115230       1        1  37.928398
2       0.648955  71.707100       2        1  17.258455
3      71.019684   9.971006       3        1   8.976490
6       3.785981  33.185135       4        1   8.339636
1     -42.666500  -9.222070       5        1   8.129066
4      -3.621689 -43.526516       6        1   7.578809
9      -3.632402  -4.171708       7        1   4.866623
8     -36.584400  35.804413       8        1   3.702746
0      44.391380  50.615383       9        1   1.695241
7      40.235809 -31.591665      10        1   1.524537, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
125   Default  4.000000       privacy  4.000000  30.0000  30.0000
14    Default  3.000000     assessing  3.000000  29.0000  29.0000
64    Default  6.000000    experience  6.000000  28.0000  28.0000
143   Default  3.000000      relevant  3.000000  27.0000  27.0000
85    Default  4.000000   information  4.000000  26.0000  26.0000
190   Default  3.000000         years  3.000000  25.0000  25.0000
134   Default  3.000000    protection  3.000000  24.0000  24.0000
90    Default  4.000000     knowledge  4.000000  23.0000  23.0000
149   Default  3.000000      sciences  3.000000  22.0000  22.0000
133   Default  4.000000      projects  4.000000  21.0000  21.0000
188   Default  5.000000       working  5.000000  20.0000  20.0000
53    Default  3.000000     effective  3.000000  19.0000  19.0000
154   Default  3.000000        skills  3.000000  18.0000  18.0000
28    Default  3.000000       clients  3.000000  17.0000  17.0000
31    Default  4.000000       complex  4.000000  16.0000  16.0000
130   Default  3.000000     producing  3.000000  15.0000  15.0000
88    Default  4.000000           key  4.000000  14.0000  14.0000
60    Default  3.000000         equip  3.000000  13.0000  13.0000
40    Default  5.000000      customer  5.000000  12.0000  12.0000
16    Default  3.000000    background  3.000000  11.0000  11.0000
110   Default  4.000000           non  4.000000  10.0000  10.0000
136   Default  3.000000        public  3.000000   9.0000   9.0000
102   Default  3.000000     marketing  3.000000   8.0000   8.0000
155   Default  3.000000        social  3.000000   7.0000   7.0000
174   Default  4.000000     technical  4.000000   6.0000   6.0000
73    Default  4.000000    governance  4.000000   5.0000   5.0000
99    Default  3.000000       looking  3.000000   4.0000   4.0000
172   Default  4.000000          team  4.000000   3.0000   3.0000
32    Default  3.000000    compliance  3.000000   2.0000   2.0000
157   Default  4.000000     solutions  4.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
189   Topic10  0.069607          year  3.691164   0.2126  -5.1420
36    Topic10  0.058284    consulting  3.099726   0.2097  -5.3195
5     Topic10  0.062384     analysing  3.328225   0.2066  -5.2515
83    Topic10  0.069422          hmrc  3.712763   0.2042  -5.1446
134   Topic10  0.064357    protection  3.479219   0.1934  -5.2204
161   Topic10  0.065492          stem  3.542635   0.1928  -5.2029
124   Topic10  0.069105    principles  3.742489   0.1916  -5.1492
180   Topic10  0.064520          turn  3.501750   0.1894  -5.2179
55    Topic10  0.073180    engagement  3.979828   0.1874  -5.0919
7     Topic10  0.063947      analytic  3.478079   0.1873  -5.2268
110   Topic10  0.073880           non  4.036407   0.1828  -5.0824
29    Topic10  0.059078    commercial  3.237790   0.1797  -5.3060
111   Topic10  0.067245            oc  3.691882   0.1779  -5.1765
158   Topic10  0.063235       sources  3.495327   0.1712  -5.2380
102   Topic10  0.064632     marketing  3.575270   0.1704  -5.2161
133   Topic10  0.073978      projects  4.156074   0.1549  -5.0811
50    Topic10  0.071679   development  4.030899   0

### Top 10 words per topic Using TFIDF

In [94]:
for index,topic in enumerate(LDA2.components_):
    print(f'THE TOP {n_top_words} WORDS FOR TOPIC #{index}')
    print([tfidf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-n_top_words:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['position', 'skills', 'work', 'looking', 'career', 'time', 'project', 'management', 'business', 'manager', 'experience', 'team', 'customer', 'aflac', 'sales']


THE TOP 15 WORDS FOR TOPIC #1
['strong', 'continually', 'retailer', 'skills', 'verizon', 'excellent', 'management', 'experience', 'time', 'store', 'products', 'retail', 'sales', 'wireless', 'gowireless']


THE TOP 15 WORDS FOR TOPIC #2
['central', 'technicians', 'distributor', 'business', 'mechanics', 'reliably', 'inventory', 'mobile', 'customers', 'tools', 'payment', 'outside', 'equipment', 'sales', 'matco']


THE TOP 15 WORDS FOR TOPIC #3
['controlling', 'relocation', 'stepping', 'plenty', 'men', 'women', 'home', 'tools', 'growth', 'calls', 'list', 'protected', 'business', 'snap', 'franchise']


THE TOP 15 WORDS FOR TOPIC #4
['characteristic', 'veteran', 'disability', 'age', 'marital', 'regard', 'religion', 'sex', 'race', 'origin', 'color', 'maintenance', 'hands', 'status', 'aerotek']


THE TOP 

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [95]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(LDA2, tfidf, tfidf_vectorizer, mds='tsne')
dash

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0     -29.594002  60.112713       1        1  60.970431
6     -45.996586  11.891685       2        1  10.729470
2      20.368856  70.019325       3        1   9.971675
1      53.928768  31.704309       4        1   7.025038
3     -12.437127 -26.423267       5        1   5.589067
4       3.965971  21.797720       6        1   3.769257
5      37.524284 -16.517805       7        1   1.945061, topic_info=     Category        Freq                  Term       Total  loglift  logprob
term                                                                         
4334  Default  191.000000                   tax  191.000000  30.0000  30.0000
2527  Default  166.000000               liberty  166.000000  29.0000  29.0000
1900  Default  159.000000             franchise  159.000000  28.0000  28.0000
3847  Default  337.000000                 sales  337.000000  27.0000  27.0000
654   Default  227.000000              business  227.000000  26.0000  26.0000
2692  Default  117.000000                 matco  117.000000  25.0000  25.0000
2000  Default   93.000000            gowireless   93.000000  24.0000  24.0000
4784  Default   78.000000              wireless   78.000000  23.0000  23.0000
4042  Default   70.000000                  snap   70.000000  22.0000  22.0000
1620  Default   99.000000             equipment   99.000000  21.0000  21.0000
3049  Default   86.000000               outside   86.000000  20.0000  20.0000
3310  Default   89.000000           preparation   89.000000  19.0000  19.0000
4462  Default  106.000000                 tools  106.000000  18.0000  18.0000
3133  Default   76.000000               payment   76.000000  17.0000  17.0000
2978  Default   92.000000               ongoing   92.000000  16.0000  16.0000
2493  Default   83.000000                 learn   83.000000  15.0000  15.0000
1187  Default  137.000000             customers  137.000000  14.0000  14.0000
3764  Default   82.000000                retail   82.000000  13.0000  13.0000
259   Default   43.000000               aerotek   43.000000  12.0000  12.0000
4280  Default  126.000000               support  126.000000  11.0000  11.0000
676   Default   78.000000                 calls   78.000000  10.0000  10.0000
4185  Default  117.000000                 store  117.000000   9.0000   9.0000
2559  Default   60.000000                  list   60.000000   8.0000   8.0000
910   Default   80.000000             community   80.000000   7.0000   7.0000
3448  Default   52.000000             protected   52.000000   6.0000   6.0000
2884  Default  123.000000                  need  123.000000   5.0000   5.0000
2814  Default   79.000000                mobile   79.000000   4.0000   4.0000
2032  Default   67.000000                growth   67.000000   3.0000   3.0000
2351  Default   80.000000             inventory   80.000000   2.0000   2.0000
3901  Default   56.000000              seasonal   56.000000   1.0000   1.0000
...       ...         ...                   ...         ...      ...      ...
2290   Topic7    4.733537         insurancemust    5.645133   3.7638  -5.1941
381    Topic7    4.733492            appraising    5.645352   3.7637  -5.1941
1615   Topic7    4.710135     environmentstrong    5.690052   3.7509  -5.1991
4668   Topic7    8.519819              vehicles   10.371851   3.7432  -4.6064
47     Topic7    4.099984                   25k    5.002328   3.7410  -5.3378
4667   Topic7   15.269387               vehicle   18.890255   3.7271  -4.0229
283    Topic7    3.644737                ageuse    4.525715   3.7234  -5.4555
2581   Topic7    3.003784               lobster    3.870263   3.6864  -5.6489
2656   Topic7    2.755770        mannerspecific    3.613312   3.6689  -5.7351
4262   Topic7    7.382296               sundays    9.680589   3.6688  -4.7497
167    Topic7    2.755369           accountpaid    3.613782   3.6687  -5.7352
4495   Topic7    2.75

## NMF modelling

##### Fit NMF Model  - Frobenius norm

In [139]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
sf.print_top_words(nmf, tfidf_feature_names, n_top_words)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=5000 and n_features=5000...
done in 1.397s.

Topics in NMF model (Frobenius norm):

The top 15 words for topic #0 are:tax liberty preparation ongoing learn support franchisees seasonal community year market industry marketing training sluggish

The top 15 words for topic #1 are:matco equipment sales tools outside inventory customers reliably mechanics distributor technicians automotive payment tool business

The top 15 words for topic #2 are:vector regular time opportunities income looking growing need people merit cutco needing tenure cut doesn

The top 15 words for topic #3 are:franchise snap business protected list calls tools growth stepping plenty men women controlling franchises celebrating

The top 15 words for topic #4 are:aflac career sales programs nation need representative balance insurance provides community companies employees team level

The top 15 words for topic #5 are:gowireless wireless sales reta

##### Fit NMF Model  - Generalized Kullback-Leibler divergence

In [140]:
# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")

# User TFIDF vectorizer
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Print top words
sf.print_top_words(nmf, tfidf_feature_names, n_top_words)


/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=5000 and n_features=5000...
done in 4.372s.

Topics in NMF model (generalized Kullback-Leibler divergence):

The top 15 words for topic #0 are:tax liberty preparation ongoing support learn seasonal franchisees industry community year market knowledge weekly stands

The top 15 words for topic #1 are:sales matco store equipment customers tools outside use inventory business distributor technicians success mechanics reliably

The top 15 words for topic #2 are:time looking opportunities training income growing regular vector management need team people professional want year

The top 15 words for topic #3 are:franchise snap business protected list calls growth tools relocation home plenty men controlling women stepping

The top 15 words for topic #4 are:aflac career sales programs nation insurance need balance representative provides employees community companies team work

The top 15 words for 

### LDA

##### Fit Model with TF (DTM) vectorizer

In [141]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
# Fit using TF vectorizer
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
sf.print_top_words(lda, tf_feature_names, n_top_words)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fitting LDA models with tf features, n_samples=5000 and n_features=5000...
done in 6.965s.

Topics in LDA model:

The top 15 words for topic #0 are:tool list tools snap protected future calls place takes start people equipment available independent benefits

The top 15 words for topic #1 are:time looking opportunities need training team people growing management regular vector income career want options

The top 15 words for topic #2 are:gowireless sales wireless products retail experience management time store services skills strong excellent year customer

The top 15 words for topic #3 are:business franchise management sales training opportunity ll based manager service skills plan aspects ranked day

The top 15 words for topic #4 are:customer experience company service skills work years position required opportunity store benefits career office compensation

The top 15 words for topic #5 are:manager time sales business franchise commercial operations extra customer management servic

In [142]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')
dash

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952   8.115230       1        1  25.357801
6       0.648955  71.707100       2        1  14.174462
4      71.019684   9.971006       3        1  11.549103
2       3.785981  33.185135       4        1  11.013260
8     -42.666500  -9.222070       5        1  10.989082
9      -3.621689 -43.526516       6        1   8.184466
5      -3.632402  -4.171708       7        1   6.701834
0     -36.584400  35.804413       8        1   4.152514
3      44.391380  50.615383       9        1   4.023279
1      40.235809 -31.591665      10        1   3.854198, topic_info=     Category          Freq           Term         Total  loglift  logprob
term                                                                      
678   Default   8731.000000       business   8731.000000  30.0000  30.0000
3963  Default  14516.000000          sales  14516.000000  29.0000  29.0000
1970  Default   3492.000000      franchise   3492.000000  28.0000  28.0000
4480  Default   3313.000000            tax   3313.000000  27.0000  27.0000
2737  Default   3775.000000        manager   3775.000000  26.0000  26.0000
2736  Default   5060.000000     management   5060.000000  25.0000  25.0000
2615  Default   2713.000000        liberty   2713.000000  24.0000  24.0000
277   Default   2617.000000          aflac   2617.000000  23.0000  23.0000
2074  Default   2434.000000     gowireless   2434.000000  22.0000  22.0000
4584  Default   4477.000000           time   4477.000000  21.0000  21.0000
4945  Default   2061.000000       wireless   2061.000000  20.0000  20.0000
4649  Default   3257.000000       training   3257.000000  19.0000  19.0000
4494  Default   5304.000000           team   5304.000000  18.0000  18.0000
4614  Default   2357.000000          tools   2357.000000  17.0000  17.0000
2688  Default   3567.000000        looking   3567.000000  16.0000  16.0000
2986  Default   2825.000000           need   2825.000000  15.0000  15.0000
1219  Default   4393.000000      customers   4393.000000  14.0000  14.0000
3101  Default   2796.000000  opportunities   2796.000000  13.0000  13.0000
3882  Default   2623.000000         retail   2623.000000  12.0000  12.0000
4419  Default   3923.000000        support   3923.000000  11.0000  11.0000
3259  Default   2088.000000         people   2088.000000  10.0000  10.0000
4155  Default   3420.000000         skills   3420.000000   9.0000   9.0000
1782  Default   4930.000000     experience   4930.000000   8.0000   8.0000
3102  Default   3503.000000    opportunity   3503.000000   7.0000   7.0000
1672  Default   1723.000000      equipment   1723.000000   6.0000   6.0000
3085  Default   1797.000000        ongoing   1797.000000   5.0000   5.0000
2658  Default   1543.000000             ll   1543.000000   4.0000   4.0000
727   Default   3580.000000         career   3580.000000   3.0000   3.0000
2782  Default   1602.000000          matco   1602.000000   2.0000   2.0000
3433  Default   1700.000000    preparation   1700.000000   1.0000   1.0000
...       ...           ...            ...           ...      ...      ...
3541  Topic10    266.858815      promotion    359.152341   2.9590  -4.6793
4284  Topic10    252.551106        started    340.910642   2.9560  -4.7344
1084  Topic10    267.857028      continued    373.384446   2.9239  -4.6755
4821  Topic10    316.042095             ve    473.524963   2.8517  -4.5101
4889  Topic10    325.353998           want    514.475330   2.7978  -4.4811
3101  Topic10    753.397378  opportunities   2796.544004   1.9445  -3.6414
4584  Topic10    945.708488           time   4477.848153   1.7010  -3.4141
2688  Topic10    822.671333        looking   3567.123251   1.7890  -3.5534
2105  Topic10    506.715158        growing   1594.663846   2.1095  -4.0380
3259  Topic10    528.057477         people   2088.222121   1.8811  -3.9968
2986  Topic10    587.303366           need   2825.375201   1

#### Fit Model with TfIDF vectorizer


In [143]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()

# Fit using TFIDF vectorizer
lda.fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
sf.print_top_words(lda, tfidf_feature_names, n_top_words)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fitting LDA models with tf features, n_samples=5000 and n_features=5000...
done in 3.274s.

Topics in LDA model:

The top 15 words for topic #0 are:aldi cashiers cashier riverside entrusted advantages maintenance merchandising electrical 12 project lexington lobster technician shift

The top 15 words for topic #1 are:baker tilly krause virchow office needed evening shipping direct trouble wi message repairing shooting perform

The top 15 words for topic #2 are:muzak project csr process accordance manager transmission engineering processes director leader business improvement channels engineer

The top 15 words for topic #3 are:tcs tata consultancy accelerators estimator designers focuses junit cultural near network java project chicago global

The top 15 words for topic #4 are:jewelry heartland distance ocean pacific coast oregon woodbridge administrative branch benefit office postion dunbar live

The top 15 words for topic #5 are:lobster red chrysler dodge jeep metropolitan representa

In [144]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer, mds='tsne')
dash

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9      32.420952   8.115230       1        1  55.889314
6       0.648955  71.707100       2        1  17.502839
8      71.019684   9.971006       3        1  13.350387
7       3.785981  33.185135       4        1   7.023228
5     -42.666500  -9.222070       5        1   1.088102
3      -3.621689 -43.526516       6        1   1.036290
0      -3.632402  -4.171708       7        1   1.033284
1     -36.584400  35.804413       8        1   1.027598
4      44.391380  50.615383       9        1   1.024875
2      40.235809 -31.591665      10        1   1.024083, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
4480  Default  194.000000           tax  194.000000  30.0000  30.0000
2615  Default  169.000000       liberty  169.000000  29.0000  29.0000
2782  Default  130.000000         matco  130.000000  28.0000  28.0000
3963  Default  349.000000         sales  349.000000  27.0000  27.0000
2074  Default   91.000000    gowireless   91.000000  26.0000  26.0000
4945  Default   76.000000      wireless   76.000000  25.0000  25.0000
1672  Default  106.000000     equipment  106.000000  24.0000  24.0000
1219  Default  146.000000     customers  146.000000  23.0000  23.0000
3158  Default   93.000000       outside   93.000000  22.0000  22.0000
3246  Default   84.000000       payment   84.000000  21.0000  21.0000
4321  Default  126.000000         store  126.000000  20.0000  20.0000
2912  Default   88.000000        mobile   88.000000  19.0000  19.0000
4614  Default  109.000000         tools  109.000000  18.0000  18.0000
3433  Default   90.000000   preparation   90.000000  17.0000  17.0000
3882  Default   89.000000        retail   89.000000  16.0000  16.0000
3085  Default   90.000000       ongoing   90.000000  15.0000  15.0000
1782  Default  133.000000    experience  133.000000  14.0000  14.0000
3509  Default   91.000000      products   91.000000  13.0000  13.0000
2579  Default   85.000000         learn   85.000000  12.0000  12.0000
2423  Default   85.000000     inventory   85.000000  11.0000  11.0000
3767  Default   52.000000      reliably   52.000000  10.0000  10.0000
2816  Default   52.000000     mechanics   52.000000   9.0000   9.0000
1452  Default   54.000000   distributor   54.000000   8.0000   8.0000
4988  Default   95.000000          year   95.000000   7.0000   7.0000
4504  Default   53.000000   technicians   53.000000   6.0000   6.0000
678   Default  229.000000      business  229.000000   5.0000   5.0000
4155  Default  106.000000        skills  106.000000   4.0000   4.0000
768   Default   50.000000       central   50.000000   3.0000   3.0000
2737  Default  132.000000       manager  132.000000   2.0000   2.0000
1972  Default   57.000000   franchisees   57.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
1548  Topic10    0.089455        effect    1.331082   1.8814  -8.5173
3605  Topic10    0.089536       pursuit    1.456973   1.7919  -8.5164
4229  Topic10    0.089438      specials    1.316153   1.8924  -8.5175
3294  Topic10    0.089438          pest    1.318294   1.8908  -8.5175
1109  Topic10    0.089563    convenient    1.519372   1.7503  -8.5161
3532  Topic10    0.091538       project   96.454459  -2.3787  -8.4943
3495  Topic10    0.091063       process   35.892016  -1.3953  -8.4995
155   Topic10    0.090397    accordance   10.083866  -0.1331  -8.5069
2737  Topic10    0.090287       manager  132.687254  -2.7114  -8.5081
1622  Topic10    0.090123   engineering   13.726117  -0.4445  -8.5099
3496  Topic10    0.090096     processes   16.656497  -0.6383  -8.5102
1408  Topic10    0.090085      director   35.581284  -1.3974  -8.5103
2573  Topic10    0.090053        leader   23.229744  -0.9714  -8.5107
678   Topic10    0.090031      business  229.679562  -3.2629  -8.

<div class="alert alert-block alert-info">
<b>Credits and References</b> 
    
This notebook has leveraged work from the following authors and adjusted it accordingly to fit the specific dataset under BSD 3 Clause License:


**Authors** 
* Olivier Grisel olivier.grisel@ensta.org
* Lars Buitinck
* Chyi-Kwei Yau chyikwei.yau@gmail.com>
* License: BSD 3 clause

See link below for full Example of their work:

https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

    
</div>

### Map topic in dataframe for Job Description

In [145]:
k_jobs['Job Description'][0]

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'This Administrative Assistant position is responsible for performing a variety of clerical and administrative support functions in the areas of communications, data entry and retrieval, typing, filing, copying, and coordination of related administrative activities. JOB REQUIREMENTS:Attention to detailAbility to work in a fast paced environmentInvoicingProficient in Word, Excel and OutlookOther Administrative responsibilities as needed.QUALIFICATIONS:High School DiplomaAdministrative Assistant Experience1-2 Years work experienceWorking knowledge of all basic office machines (fax, copier, 10key, etc.)Strong verbal and written communication skillsStrong work ethic.'

In [166]:
# Topic distribution for Job[0] rounded to three decimal points
topic_results = lda.transform(tfidf).round(3)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [167]:
k_jobs['Job Description'][25]

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"Navistar International Corporation is a holding company whose subsidiaries and affiliates produce International® brand commercial and military trucks, MaxxForce® brand diesel engines, IC Bus™ brand school and commercial buses, Monaco® RV brands of recreational vehicles, and Workhorse® brand chassis for motor homes and step vans.  It also is a private-label designer and manufacturer of diesel engines for the pickup truck, van and SUV markets. The company provides truck and diesel engine service parts and also offers financing services through one of its affiliates. International Truck Formed in 1902, International is a leading producer of medium trucks, heavy trucks, severe service vehicles. Our products, parts and services are sold through a network of nearly 1,000 dealer outlets in the United States, Canada, Brazil, and Mexico and more than 60 dealers in 90 countries throughout the world.Oversees a small project or phases of a larger project. Responsible for coordinating activities o

In [168]:
topic_results[6]

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012, 0.012,
       0.895])

In [169]:
# Get index position of highest probability
topic_results[0].argmax()

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


9

In [153]:
# Create a new copy of the original dataframe with only the N Samples modelling took place for
modelled_k_jobs = k_jobs.iloc[0:n_samples].copy()

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [154]:
modelled_k_jobs.head()

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Location,Company,Job Title,Job Description,Salary,E-mail
0,,None,Administrative Assistant,This Administrative Assistant position is resp...,,
1,,,Administrative Assistant,ADMINISTRATIVE ASSISTANT Part Time The West Or...,,Kenneth.harker@healthcentral.org
2,,americanwest,Administrative Assistant,Administrative Assistant - In Bus 26yrs Fashi...,,barnard@americanwest.cc
3,,None,Sales Representative,Are you ready for something new? Are you tired...,,
4,,None,Customer Service Representative,Superior Staff Resources is currently seeking ...,,


In [155]:
# Map model topics to N Samples
modelled_k_jobs['Topic'] = topic_results.argmax(axis=1)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [159]:
modelled_k_jobs.tail(50)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Location,Company,Job Title,Job Description,Salary,E-mail,Topic
4950,,None,Administrative Assistant,Basic Accounting (Income statements & Balance ...,,,9
4951,,None,Account Executive,Range Inc. is looking for a high energy profes...,,,9
4952,,None,Staff Accountant,SUMMARY This position is a key support role i...,,,9
4953,,None,Sales Representative,Sales RepresentativeSell IT (Information Techn...,,,9
4954,,None,Customer Service Representative,Folsom area company is seeking Customer Servic...,,,9
4955,,None,Staff Accountant,Looking to be part of a talented team with a w...,,,9
4956,,None,Administrative Assistant,Key responsibilities of the Administrative Ass...,,,9
4957,,None,Sales Representative,Western Services Corporation (www.ws-corp.com)...,,,9
4958,,None,Sales Representative,"Four Seasons, a company in the Indoor Tanning...",,,9
4959,,aol,Physical Therapist,A privately owned outpatient physical therapy ...,,innercirclept@aol.com,9


In [111]:
test_topics = sf.return_top_words(lda, tfidf_feature_names, n_top_words)
test_topics

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndexError: list index out of range

In [112]:
t_topics = []
for index,topic in enumerate(LDA2.components_):
    t_topics.append([tfidf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-n_top_words:]])

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [113]:
print(t_topics[0])

['position', 'skills', 'work', 'looking', 'career', 'time', 'project', 'management', 'business', 'manager', 'experience', 'team', 'customer', 'aflac', 'sales']


/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [114]:
t_topics

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['position',
  'skills',
  'work',
  'looking',
  'career',
  'time',
  'project',
  'management',
  'business',
  'manager',
  'experience',
  'team',
  'customer',
  'aflac',
  'sales'],
 ['strong',
  'continually',
  'retailer',
  'skills',
  'verizon',
  'excellent',
  'management',
  'experience',
  'time',
  'store',
  'products',
  'retail',
  'sales',
  'wireless',
  'gowireless'],
 ['central',
  'technicians',
  'distributor',
  'business',
  'mechanics',
  'reliably',
  'inventory',
  'mobile',
  'customers',
  'tools',
  'payment',
  'outside',
  'equipment',
  'sales',
  'matco'],
 ['controlling',
  'relocation',
  'stepping',
  'plenty',
  'men',
  'women',
  'home',
  'tools',
  'growth',
  'calls',
  'list',
  'protected',
  'business',
  'snap',
  'franchise'],
 ['characteristic',
  'veteran',
  'disability',
  'age',
  'marital',
  'regard',
  'religion',
  'sex',
  'race',
  'origin',
  'color',
  'maintenance',
  'hands',
  'status',
  'aerotek'],
 ['encourage',
  '

In [115]:
top_topics = pd.DataFrame(t_topics)

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [116]:
top_topics

/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,position,skills,work,looking,career,time,project,management,business,manager,experience,team,customer,aflac,sales
1,strong,continually,retailer,skills,verizon,excellent,management,experience,time,store,products,retail,sales,wireless,gowireless
2,central,technicians,distributor,business,mechanics,reliably,inventory,mobile,customers,tools,payment,outside,equipment,sales,matco
3,controlling,relocation,stepping,plenty,men,women,home,tools,growth,calls,list,protected,business,snap,franchise
4,characteristic,veteran,disability,age,marital,regard,religion,sex,race,origin,color,maintenance,hands,status,aerotek
5,encourage,wealth,environmentstrong,appraising,insurancemust,mvrvalid,titlebucks,equityauto,liens,competitively,titlemax,sundays,vehicles,tmx,vehicle
6,taxes,training,marketing,industry,market,year,community,franchisees,seasonal,support,learn,ongoing,preparation,liberty,tax


In [117]:

#df_topic_keywords = pd.DataFrame(topic_keywords)
top_topics.columns = ['Word '+str(i) for i in range(top_topics.shape[1])]
top_topics.index = ['Topic '+str(i) for i in range(top_topics.shape[0])]
top_topics


/home/etzimopoulos/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,position,skills,work,looking,career,time,project,management,business,manager,experience,team,customer,aflac,sales
Topic 1,strong,continually,retailer,skills,verizon,excellent,management,experience,time,store,products,retail,sales,wireless,gowireless
Topic 2,central,technicians,distributor,business,mechanics,reliably,inventory,mobile,customers,tools,payment,outside,equipment,sales,matco
Topic 3,controlling,relocation,stepping,plenty,men,women,home,tools,growth,calls,list,protected,business,snap,franchise
Topic 4,characteristic,veteran,disability,age,marital,regard,religion,sex,race,origin,color,maintenance,hands,status,aerotek
Topic 5,encourage,wealth,environmentstrong,appraising,insurancemust,mvrvalid,titlebucks,equityauto,liens,competitively,titlemax,sundays,vehicles,tmx,vehicle
Topic 6,taxes,training,marketing,industry,market,year,community,franchisees,seasonal,support,learn,ongoing,preparation,liberty,tax


In [106]:
pp

### Predict topic for Unseen Job Description